labels.npy corrisponde alla struttura che contiene le info di classe, oggetto e posizione sulla griglia. Nel caso del codice che ho su github, 

 0 = row_id, 1 = col_id, 2 = class_id, colonna 3 = object_id

In [1]:
import numpy as np
from sklearn.utils import shuffle
from utils import *

data = np.load('SITS-Missing-Data/D1_balaruc_samples.npy')
masks = np.load('SITS-Missing-Data/D2_balaruc_masks.npy')
lut = np.load('SITS-Missing-Data/D3_balaruc_lut.npy')

In [2]:
data.shape, masks.shape, lut.shape

((137606, 54, 16), (137606, 54), (137606, 4))

In [12]:
lut[:,3]#, np.unique(, return_counts=True)

array([704, 701, 702, ..., 253, 254, 255])

In [53]:
def get_unique_ids_by_class(lut):
    labels = lut[:, 1]
    unique_labels = np.unique(labels)
    ids_by_class = {}
    for label in unique_labels:
      idx = np.where(labels == label)
      lut_subset = lut[idx]
      ids_by_class[label] = np.unique(lut_subset[:, 0])
    return ids_by_class

def get_idx_of_object_ids(ids, lut):
    lut_ids = lut[:, 0]
    tot_idx = []
    for i in ids:
        tot_idx.append(np.where(lut_ids == i)[0])
    tot_idx = np.concatenate(tot_idx, axis=0)
    return tot_idx

def get_split_idx(lut, train_perc=.6, val_perc=.2):
  train_idx, valid_idx, test_idx = [], [], []
  unique_ids_by_class = get_unique_ids_by_class(lut)

  for label in unique_ids_by_class:
    ids = unique_ids_by_class[label]
    ids = shuffle(ids)
    
    limit_train = int(len(ids)* train_perc )
    limit_val = limit_train + int(len(ids)* val_perc)
    
    train_idx.extend(get_idx_of_object_ids(ids[0:limit_train], lut))
    valid_idx.extend(get_idx_of_object_ids(ids[limit_train:limit_val], lut))
    test_idx.extend(get_idx_of_object_ids(ids[limit_val::], lut))
  return (train_idx,), (valid_idx,), (test_idx,)

train_idx, val_idx, test_idx = get_split_idx(lut)

In [54]:
unique_ids_by_class = get_unique_ids_by_class(lut)

In [64]:
a = 0
for c, ids in unique_ids_by_class.items():
  print(c, len(ids))
  a += len(ids)
print(a)

0 712
1 328
2 82
3 172
4 223
5 32
6 39
7 56
1644


In [3]:
data[np.where(masks == 1)] = MISSING_VALUE

num_steps = data.shape[1]
num_bands = data.shape[2]
labels, num_classes = transfer_labels(lut[:, 1])
# labels = convert_to_one_hot(labels, num_classes=len(np.unique(labels)))
prediction_target = data[:, 1:]
mask = np.ones_like(prediction_target)
mask[np.where(prediction_target == MISSING_VALUE)] = 0

In [4]:
check_missing_ratio(mask)

MISSING RATIO: 0.339


In [7]:
for idx in [train_idx, val_idx, test_idx]:
  check_missing_ratio(np.array(mask[idx]))

MISSING RATIO: 0.339
MISSING RATIO: 0.337
MISSING RATIO: 0.341


In [65]:
object_ids = lut[:,0]

In [66]:
object_ids.shape

(137606,)

In [67]:
a, b = 0, 0
for l in np.unique(labels):
  lab = l+1
  pixels =  np.where(labels == l)[0].shape[0]
  p = len(np.unique(object_ids[np.where(labels == l)]))
  a += pixels
  b += p
  print(lab, p, pixels)
print(a, b)

1 712 10412
2 328 5165
3 82 32095
4 172 49175
5 223 28667
6 32 2075
7 39 5205
8 56 4812
137606 1644


In [20]:
len(np.unique(object_ids))

999

In [26]:
a,b = np.unique(object_ids, return_counts=True)

In [45]:
idx5 = np.where(labels[test_idx] == 5)

In [ ]:
object_ids[idx5]

In [42]:
np.where(labels[test_idx] == 5)[0].shape, labels.shape

((220,), (137606,))

In [41]:
test_labels = labels[test_idx]
np.where(test_labels == 5)[0].shape, test_labels.shape

((220,), (29402,))

In [81]:
tota, totb = 0,0
for idx in [train_idx, val_idx, test_idx]:
  print("----")
  a,b = 0,0
  for l in np.unique(labels):
    lab = l+1
    pixels =  np.where(labels[idx] == l)[0].shape[0]
    polygons = len(np.unique(object_ids[idx][np.where(labels[idx] == l)]))
    percentage_of_pixels = round(((pixels / labels[idx].shape[0])*100),2)
    print(f"{lab} & \t {polygons} & \t {pixels} ({percentage_of_pixels:.2f}\%)")
    a += pixels
    b += polygons     
  print(b, a)

  

----
1 & 	 427 & 	 6486 (7.53\%)
2 & 	 196 & 	 3369 (3.91\%)
3 & 	 49 & 	 21298 (24.73\%)
4 & 	 103 & 	 30631 (35.56\%)
5 & 	 133 & 	 16933 (19.66\%)
6 & 	 19 & 	 1428 (1.66\%)
7 & 	 23 & 	 3399 (3.95\%)
8 & 	 33 & 	 2595 (3.01\%)
983 86139
----
1 & 	 142 & 	 2266 (8.84\%)
2 & 	 65 & 	 803 (3.13\%)
3 & 	 16 & 	 5308 (20.70\%)
4 & 	 34 & 	 8194 (31.95\%)
5 & 	 44 & 	 6362 (24.81\%)
6 & 	 6 & 	 309 (1.20\%)
7 & 	 7 & 	 948 (3.70\%)
8 & 	 11 & 	 1455 (5.67\%)
325 25645
----
1 & 	 143 & 	 1660 (6.43\%)
2 & 	 67 & 	 993 (3.85\%)
3 & 	 17 & 	 5489 (21.26\%)
4 & 	 35 & 	 10350 (40.08\%)
5 & 	 46 & 	 5372 (20.80\%)
6 & 	 7 & 	 338 (1.31\%)
7 & 	 9 & 	 858 (3.32\%)
8 & 	 12 & 	 762 (2.95\%)
336 25822


In [45]:
for idx in [train_idx, val_idx, test_idx]:
  l = lut[idx]
  print(np.unique(l[:, 0])[0:20])

[ 2  3  5  7  8  9 10 11 13 14 15 16 17 19 21 23 25 26 28 31]
[ 1  4  6 18 22 24 27 29 30 36 43 44 47 49 52 61 64 68 71 73]
[ 0 12 20 34 38 39 42 46 48 54 66 69 74 76 85 95 96 97 98 99]
